In [75]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable


In [76]:
import os
import openai
import shutil

In [77]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [78]:
!pip install Gitpython

Defaulting to user installation because normal site-packages is not writeable


In [79]:
from git import Repo

In [80]:
from pathlib import Path

In [81]:
#Just testing printing working directory

In [82]:
pwd

'C:\\Users\\david\\Desktop\\Python\\Courses\\Udemy\\OpenAI Python Bootcamp\\dawidahs.github.io'

In [83]:
PATH_TO_BLOG_REPO = Path('C:\\Users\\david\\Desktop\\Python\\Courses\\Udemy\\OpenAI Python Bootcamp\\dawidahs.github.io\\.git')

In [84]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [85]:
PATH_TO_CONTENT = PATH_TO_BLOG/'content'

In [86]:
PATH_TO_CONTENT

WindowsPath('C:/Users/david/Desktop/Python/Courses/Udemy/OpenAI Python Bootcamp/dawidahs.github.io/content')

In [87]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [88]:
def update_blog(commit_message="Updates blog"):
    repo = Repo(PATH_TO_BLOG_REPO)
    repo.git.add(all=True)
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()

In [89]:
#random_text_string = 'smlkmssdlsdvdsdvsdmsdl'

In [90]:
#with open(PATH_TO_BLOG/'index.html', 'w') as f:
    #f.write(random_text_string)

In [91]:
 #Test push an update

In [92]:
#update_blog()

In [93]:
def create_new_blog(title, content, cover_image):
    cover_image = Path(cover_image)
    
    files = len(list(PATH_TO_CONTENT.glob('*.html')))
    new_title = f'{files+1}.html'
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image, PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        with open(path_to_new_content, 'w') as f:
            f.write('<DOCTYPE HTML>\n')
            f.write('<html>\n')
            f.write('<head>\n')
            f.write(f'<title> {title} </title>\n')
            f.write('</head>\n')
            
            f.write('<body>\n')
            f.write(f"<image src='{cover_image.name}' alt='Cover Image'>\n")
            f.write(f'<h1> {title} </h1>\n')
            f.write(content.replace("\n", "<br />\n"))
            f.write('</body>\n')
            
            f.write('</html>\n')
            print('Blog Created')
            return(path_to_new_content)
    else:
        raise FileExistsError('File already exists, please check you name.')

In [94]:
path_to_new_content = create_new_blog('Test Title','aklfakssadansdlasn','test.jpg')

Blog Created


In [95]:
from bs4 import BeautifulSoup as Soup

In [96]:
with open(PATH_TO_BLOG/'index.html') as index:
    soup = Soup(index.read())

In [97]:
soup

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>
   Document
  </title>
</head>
<body>
<h1>
   My Blog Home Page
  </h1>
<a href="index.html">
   Home
  </a>
<a href="content\1.html">
</a>
<a href="content\2.html">
</a>
<a href="content\3.html">
</a>
<a href="content\4.html">
</a>
</body>
</html>

In [98]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get('href'))for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls

In [99]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/'index.html') as index:
        soup = Soup(index.read())
        
    links = soup.find_all('a')
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError('Link already exist!')
        
    link_to_new_blog = soup.new_tag('a', href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog_string = path_to_new_content.name.split('.')[0]
    last_link.insert_after(link_to_new_blog)
        
    with open(PATH_TO_BLOG/'index.html', 'w') as f:
        f.write(str(soup.prettify(formatter="html")))

In [100]:
write_to_index(path_to_new_content)

In [101]:
update_blog()

In [102]:
def create_prompt(title):
    prompt =  """
    Biography:
    My name is David
    
    Blog
    Title: {}
    tags: tech, python, coding, AI, HR
    Summary: I am learning machine learning
    Full Text: """.format(title)
    return prompt

In [103]:
title = "David Python blog lesson"
print(create_prompt(title))


    Biography:
    My name is David
    
    Blog
    Title: David Python blog lesson
    tags: tech, python, coding, AI, HR
    Summary: I am learning machine learning
    Full Text: 


In [ ]:
response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", 
                   "content": create_prompt(title)}
                 ],
        temperature=0.7,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0)

In [ ]:
blog_content = response.choices[0].message.content

In [ ]:
print(blog_content)

In [ ]:
def dalle2_prompt(title):
    prompt = f"abstract image, no text, in the sytle of a comic of {title}"
    return prompt

In [ ]:
image_prompt = dalle2_prompt(title)

In [ ]:
image_prompt

In [ ]:
response = openai.Image.create(prompt=image_prompt,
                              n=1,
                              size="1024x1024")

In [ ]:
image_url = response['data'][0]['url']

In [ ]:
image_url

In [ ]:
import requests

In [ ]:
def save_image(image_url, file_name):
    image_res = requests.get(image_url, stream = True)
        
    if image_res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(image_res.raw, f)
    else:
        print("Error downloading image!")
    return image_res.status_code

In [ ]:
save_image(image_url,file_name="title2.png")

In [ ]:
from PIL import Image

In [ ]:
Image.open('title2.png')

In [ ]:
path_to_new_content = create_new_blog(title, blog_content, 'title2.png')

In [ ]:
write_to_index(path_to_new_content)

In [ ]:
update_blog()